In [1]:
import os

In [2]:
path = {"Train": os.path.join("F:/Research\Bee/20k_audio_splitted_dataset/train"),
        "Val": os.path.join("F:/Research\Bee/20k_audio_splitted_dataset/val"),
        "Test": os.path.join("F:/Research\Bee/20k_audio_splitted_dataset/test"),
       }

In [3]:
import pandas as pd
import seaborn as sns
import wave
import librosa
import numpy as np
import scipy
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import IPython.display as ipd
import time

In [4]:
example_file = os.path.join(path['Train'], "Queen", "queen_0.wav")
example_wav = wave.open(example_file)
print("Sampling (frame) rate = ", example_wav.getframerate())
print("Total samples (frames) = ", example_wav.getnframes())
print("Duration = ", example_wav.getnframes()/example_wav.getframerate())

ipd.Audio(example_file)

Sampling (frame) rate =  16000
Total samples (frames) =  32000
Duration =  2.0


In [5]:
class Config(object):
    def __init__(self,
                 sampling_rate=44100, audio_duration=2, n_mfcc=40):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_mfcc = n_mfcc
        self.audio_length = self.sampling_rate * self.audio_duration
        self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)

In [6]:
config = Config(sampling_rate=16000, audio_duration=2, n_mfcc=40)

In [7]:
sample_data, _ = librosa.core.load(example_file, sr=16000, res_type="kaiser_fast")
mfcc_sample_data = librosa.cqt(y=sample_data, sr=16000)

In [8]:
mfcc_sample_data.shape

(84, 63)

In [9]:
mean_mfcc_sample_data = np.mean(mfcc_sample_data, axis=1)

In [10]:
mean_mfcc_sample_data.T.shape

(84,)

In [11]:
for _set, _path  in path.items():
    for _label in ["Queen", "Queenless"]:
        print(_set, "\t", _label, "has", "\t", len(os.listdir(os.path.join(_path, _label))), "files" )
         

Train 	 Queen has 	 7000 files
Train 	 Queenless has 	 7000 files
Val 	 Queen has 	 1000 files
Val 	 Queenless has 	 1000 files
Test 	 Queen has 	 2000 files
Test 	 Queenless has 	 2000 files


In [14]:
def prepare_data(fnames, config, data_dir):
    X = np.empty(shape=(len(fnames), 84))
    input_length = config.audio_length
    for i, fname in tqdm(enumerate(fnames), total=len(fnames)):
        file_path = os.path.join(data_dir, fname)
        data, _ = librosa.core.load(file_path, sr=16000, res_type="kaiser_fast")
                                                                                                                
        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")

        mfcc = librosa.cqt(y=data, sr=16000)
        mfcc_feature = np.hstack([np.mean(mfcc, axis=1)])
        X[i,] = mfcc_feature
    return X

In [15]:
t1 = time.time()
Queen_train = prepare_data(os.listdir(os.path.join(path["Train"], "Queen")), config, os.path.join(path["Train"], "Queen"))
Queenless_train = prepare_data(os.listdir(os.path.join(path["Train"], "Queenless")), config, os.path.join(path["Train"], "Queenless"))
Queen_val = prepare_data(os.listdir(os.path.join(path["Val"], "Queen")), config, os.path.join(path["Val"], "Queen"))
Queenless_val = prepare_data(os.listdir(os.path.join(path["Val"], "Queenless")), config, os.path.join(path["Val"], "Queenless"))
Queen_test = prepare_data(os.listdir(os.path.join(path["Test"], "Queen")), config, os.path.join(path["Test"], "Queen"))
Queenless_test = prepare_data(os.listdir(os.path.join(path["Test"], "Queenless")), config, os.path.join(path["Test"], "Queenless"))
t2 = time.time()

  0%|          | 0/7000 [00:00<?, ?it/s]

C:\Users\tunne\AppData\Local\Temp\ipykernel_54552\670260617.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,] = mfcc_feature


  0%|          | 0/7000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [16]:
print(f"time elapsed: {t2-t1} seconds or {int((t2-t1)//60)} minutes and {int((t2-t1)-(t2-t1)//60*60)} seconds")

time elapsed: 882.5870878696442 seconds or 14 minutes and 42 seconds


In [17]:
os.makedirs('train', mode = 0o777, exist_ok = True) 
os.makedirs('val', mode = 0o777, exist_ok = True) 
os.makedirs('test', mode = 0o777, exist_ok = True) 

In [18]:
np.save("test/train_queen", Queen_test)
np.save("test/train_queenless", Queenless_test)

In [19]:
np.save("train/train_queen", Queen_train)
np.save("train/train_queenless", Queenless_train)

In [20]:
np.save("val/train_queen", Queen_val)
np.save("val/train_queenless", Queenless_val)